In [52]:
import pandas as pd
import numpy as np
import pickle 
import dill
from tensorflow.keras.models import load_model

In [53]:
#Load the trained model, scaler pickle, onehot
model = load_model("./model.keras")

##Open the encoder and scaler
with open("label_encoder_gender.pkl", 'rb') as file:
    gender_label_encoder = pickle.load(file=file)

with open("scaler_transform.pkl", 'rb') as file:
    scaler_transform = pickle.load(file)

with open("one_hot_encoder_geo.pkl", 'rb') as file:
    one_hot_encoder_for_geo = pickle.load(file=file)

In [68]:
one_hot_encoder_for_geo.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [69]:
gender_label_encoder.classes_

array(['Female', 'Male'], dtype=object)

In [54]:
##Example input data
input_data = {
    "CreditScore": 600,
    "Geography": 'France',
    'Gender': 'Male',
    "Age": 40,
    'Tenure':3, 
    'Balance': 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}

In [55]:
one_hot_encoder_for_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [56]:
##FIRST encode geography
geo_encoded = one_hot_encoder_for_geo.transform([[input_data["Geography"]]])
geo_encoded_df = pd.DataFrame(data=geo_encoded, columns=one_hot_encoder_for_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

c:\Users\User\Documents\End-to-end ANN project\envin\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [57]:
'''geo_encoded = one_hot_encoder_for_geo.transform([input_data["Geography"]])
This is wrong:
 Right now you are giving ['France']   ❌ (1D)
 But OneHotEncoder expects data like during training:

[[France],
 [Germany],
 [Spain]]

🧠 Rule

Sklearn encoders always expect:
(samples, features)


You have:
1 sample
1 feature (Geography)

So shape must be:
(1, 1)'''

'geo_encoded = one_hot_encoder_for_geo.transform([input_data["Geography"]])\nThis is wrong:\n Right now you are giving [\'France\']   ❌ (1D)\n But OneHotEncoder expects data like during training:\n\n[[France],\n [Germany],\n [Spain]]\n\n🧠 Rule\n\nSklearn encoders always expect:\n(samples, features)\n\n\nYou have:\n1 sample\n1 feature (Geography)\n\nSo shape must be:\n(1, 1)'

In [ ]:
##Combining one hot encoded dataframe with input_data_df
input_data_df = pd.DataFrame([input_data]).reset_index(drop=True)
# print(input_data_df)
input_data_df = pd.concat([input_data_df, geo_encoded_df], axis=1)
# print(input_data_df)

input_data_df.drop(columns=['Geography'], inplace=True)


In [61]:
print(input_data_df)

   CreditScore Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          600   Male   40       3    60000              2          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1            50000               1.0                0.0   

   Geography_Spain  
0              0.0  


In [62]:
#Applying label incoder to Gender
input_data_df["Gender"] = gender_label_encoder.transform(input_data_df["Gender"])

In [64]:
##Now using standard scaler
input_data_df=scaler_transform.transform(input_data_df)

In [65]:
input_data_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [70]:
## Prediction
pred=model.predict(input_data_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [75]:
float(pred[0][0])

0.03169774264097214

In [ ]:
### Here the output is of prob cause we are applying sigmoid